In [1]:
%%writefile mapper.py
from __future__ import division
import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('stop_words_en.txt') as f:
    stop_words = set(f.read().split())

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        article_id = int(article_id)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    words = [x.lower() for x in words if x.lower() not in stop_words]
    words_set = set(words)
    
    num_of_words_in_doc = len(words)
    counts = Counter(words)
    
    for word in words_set:
        num_of_word_in_doc = counts[word]
        tf = num_of_word_in_doc / num_of_words_in_doc
        print "%s\t%d\t%f" % (word, article_id, tf)

Writing mapper.py


In [2]:
%%writefile reducer.py
from __future__ import division
import sys
from math import log

current_word = None
articles_dict = dict()

for line in sys.stdin:
    try:
        word, article_id, tf = line.strip().split('\t')
        article_id = int(article_id)
        tf = float(tf)
    except ValueError as e:
        continue
    
    if current_word != word:
        if current_word:
            idf = 1 / log(1 + len(articles_dict))
            for key_article_id, tf in articles_dict.iteritems():
                tfidf = tf * idf
                print "%s\t%d\t%f" % (current_word, key_article_id, tfidf)
        articles_dict = dict()
        current_word = word
    articles_dict[article_id] = tf

if current_word:
    print "%s\t%d\t%f" % (current_word, article_id, tfidf)

Writing reducer.py


In [10]:
%%bash

OUT_DIR="tf_idf"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming tf_idf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat tf_idf/* | grep -P 'labor\t12\t' | cut -f3

0.000351


rm: `tf_idf': No such file or directory
19/04/04 14:53:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/04/04 14:53:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/04/04 14:53:18 INFO mapred.FileInputFormat: Total input files to process : 1
19/04/04 14:53:18 INFO mapreduce.JobSubmitter: number of splits:2
19/04/04 14:53:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1554382337295_0001
19/04/04 14:53:19 INFO impl.YarnClientImpl: Submitted application application_1554382337295_0001
19/04/04 14:53:19 INFO mapreduce.Job: The url to track the job: http://b6efa8fb5d42:8088/proxy/application_1554382337295_0001/
19/04/04 14:53:19 INFO mapreduce.Job: Running job: job_1554382337295_0001
19/04/04 14:53:29 INFO mapreduce.Job: Job job_1554382337295_0001 running in uber mode : false
19/04/04 14:53:29 INFO mapreduce.Job:  map 0% reduce 0%
19/04/04 14:53:48 INFO mapreduce.Job:  map 53% reduce 0%
19/04/04 14:53:54 INFO mapreduce.Job:  m